In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install py7zr

In [ ]:
import py7zr

for dirname, _, filenames in os.walk('/kaggle/input/favorita-grocery-sales-forecasting'):
    for filename in filenames:
        archive = py7zr.SevenZipFile(os.path.join(dirname, filename), mode='r')
        archive.extractall(path="/kaggle/working")
        archive.close()

In [ ]:
from subprocess import check_output

print(check_output(["ls", "../working"]).decode("utf8"))

In [ ]:
holiday = pd.read_csv("../working/holidays_events.csv", parse_dates=['date'])
items = pd.read_csv("../working/items.csv")
oil = pd.read_csv("../working/oil.csv", parse_dates=['date'])
stores = pd.read_csv("../working/stores.csv")

train = pd.read_csv("../working/train.csv")
transactions = pd.read_csv("../working/transactions.csv", parse_dates=['date'])

In [ ]:
test = pd.read_csv("../working/test.csv")

In [ ]:
test.head()

In [ ]:
test.tail()

In [ ]:
train["date"] = pd.to_datetime(train["date"])

In [ ]:
test["date"] = pd.to_datetime(test["date"])

In [ ]:
start_date = pd.to_datetime("2016-04-16")
end_date = start_date + pd.DateOffset(months=1)

# Remote time that earthqua happen and 1 month after that 
train = train[~((train['date'] >= start_date) & (train['date'] <= end_date))]

In [ ]:
train.head()

In [ ]:
train['date'] = pd.to_datetime(train['date'])
train_filtered = train[train['date'] > '2017-01-01']

In [ ]:
def optimize_for_timeseries(df):
    # Convert 'date' to datetime
    df['date'] = pd.to_datetime(df['date'])

    # Extract time-related features
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['weekday'] = df['date'].dt.weekday
    df['quarter'] = df['date'].dt.quarter
    del df['date']

    df['id'] = df['id'].astype(np.int32)
    df['store_nbr'] = df['store_nbr'].astype(np.int8)
    df['item_nbr'] = df['item_nbr'].astype(np.uint32)
    df['unit_sales'] = df['unit_sales'].astype(np.float32)

    return df

In [ ]:
train_df = optimize_for_timeseries(train_filtered)

In [ ]:
test['unit_sales'] = 0

In [ ]:
test

In [ ]:
test_df = optimize_for_timeseries(test)

In [ ]:
test_df.isnull().sum()

In [ ]:
train_df.head()

In [ ]:
train_df["onpromotion"].fillna(-1, inplace=True)


In [ ]:
# Filter data after 2016

# Check for missing values in the filtered data
train_df.isnull().sum()


In [ ]:
holiday = holiday.loc[(holiday['transferred'] == False) & (holiday['type'] != 'Work Day')]
holiday = holiday.drop(columns=['transferred'])

holiday['type'] = holiday['type'].replace({
    'Holiday': 1,
    'Event': 2,
    'Additional': 3,
    'Transfer': 4,
    'Bridge': 5
}).fillna(0).astype(np.int8) 

In [ ]:
holiday['date'] = pd.to_datetime(holiday['date'])

holiday['year'] = pd.to_datetime(holiday['date']).dt.year.astype(np.uint16)  
holiday['month'] = pd.to_datetime(holiday['date']).dt.month.astype(np.uint8)  
holiday['day'] = pd.to_datetime(holiday['date']).dt.day.astype(np.uint8) 

del holiday['date']  
del holiday['description']

In [ ]:
train_df = train_df.merge(
    holiday[['day', 'month', 'year', 'type', 'locale']],
    on=['day', 'month', 'year'],
    how='left'
);

In [ ]:
test_df = test_df.merge(
    holiday[['day', 'month', 'year', 'type', 'locale']],
    on=['day', 'month', 'year'],
    how='left'
);

In [ ]:
train_df

In [ ]:
train_df['is_holiday'] = train_df['type'].notna().astype(np.int8);
test_df['is_holiday'] = test_df['type'].notna().astype(np.int8);

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.drop(['type', 'locale'], axis=1, inplace=True)
test_df.drop(['type', 'locale'], axis=1, inplace=True)


In [ ]:
train_df = train_df.merge(items[['item_nbr', 'family', 'class', 'perishable']], on='item_nbr', how='left')
test_df = test_df.merge(items[['item_nbr', 'family', 'class', 'perishable']], on='item_nbr', how='left')

In [ ]:
oil['date'] = pd.to_datetime(oil['date'])

oil['year'] = pd.to_datetime(oil['date']).dt.year.astype(np.uint16)  
oil['month'] = pd.to_datetime(oil['date']).dt.month.astype(np.uint8)  
oil['day'] = pd.to_datetime(oil['date']).dt.day.astype(np.uint8) 

In [ ]:
oil['dcoilwtico'] = oil['dcoilwtico'].astype(np.float32);

In [ ]:
oil


In [ ]:
# print(oil.dtypes);
# train_df = train_df.merge(oil[['dcoilwtico']], on=['year', 'month', 'day'], how='left')
# test_df = test_df.merge(oil[['dcoilwtico']], on=['year', 'month', 'day'], how='left')

In [ ]:
train_df

In [ ]:
train_df = train_df.merge(stores, on='store_nbr', how='left')

In [ ]:
test_df = test_df.merge(stores, on='store_nbr', how='left')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split



# Combine train and test data to handle features (to avoid leakage)
all_data = pd.concat([train_df, test_df], ignore_index=True)

# Convert date parts into a single datetime column for both train and test
all_data["ds"] = pd.to_datetime(all_data[["year", "month", "day"]])

# Encode categorical features (store_nbr, item_nbr) as integers (use LabelEncoder or similar)
all_data['store_nbr'] = all_data['store_nbr'].astype("category").cat.codes
all_data['item_nbr'] = all_data['item_nbr'].astype("category").cat.codes

# Define feature columns
features = ["store_nbr", "item_nbr", "onpromotion", "is_holiday", "year", "month", "weekday", "quarter"]

# Extract the target variable and features
X = all_data[features]
y = all_data["unit_sales"]

# Split into train and test sets based on the original data split
train_data = all_data[:len(train_df)]
test_data = all_data[len(train_df):]

# Separate train and test features and target
X_train = train_data[features]
y_train = train_data["unit_sales"]
X_test = test_data[features]
y_test = test_data["unit_sales"]


In [ ]:
# def normalized_weighted_rmsle(y_true, y_pred, weights):

#     log_y_true = np.log1p(y_true)
#     log_y_pred = np.log1p(y_pred)

#     log_errors = log_y_pred - log_y_true
    
#     squared_log_errors = log_errors ** 2
    
#     weighted_squared_log_errors = weights * squared_log_errors
    
#     weighted_error_sum = np.sum(weighted_squared_log_errors)
#     total_weights = np.sum(weights)
    
#     normalized_weighted_mean = weighted_error_sum / total_weights
#     return np.sqrt(normalized_weighted_mean)

# weights_train = np.where(X_train['perishable'] == 1, 1.25, 1)
# weights_test = np.where(X_test['perishable'] == 1, 1.25, 1)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

# Convert date columns into a single datetime column
train_df["ds"] = pd.to_datetime(train_df[["year", "month", "day"]])

# Encode categorical features
label_encoders = {}
for col in ["store_nbr", "item_nbr", "city", "state", "family", "type", "cluster"]:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    label_encoders[col] = le  # Store encoder for potential inverse transformation

# Load perishable weights from items.csv
items_df = pd.read_csv("items.csv")
items_df["weight"] = np.where(items_df["perishable"] == 1, 1.25, 1.00)

# Merge perishable weights with train data
train_df = train_df.merge(items_df[["item_nbr", "weight"]], on="item_nbr", how="left")

# Define feature columns
features = [
    "store_nbr", "item_nbr", "onpromotion", "is_holiday", 
    "year", "month", "weekday", "quarter", "perishable", "cluster"
]

# Extract features and target variable
X = train_df[features]
y = train_df["unit_sales"]
weights = train_df["weight"]  # Store weights

# Split into training and validation sets (80-20 split)
X_train, X_val, y_train, y_val, w_train, w_val = train_test_split(
    X, y, weights, test_size=0.2, random_state=42
)

# Use LightGBM (efficient alternative to RF)
model = LGBMRegressor(n_estimators=50, max_depth=10, learning_rate=0.05, random_state=42)
model.fit(X_train, y_train, sample_weight=w_train)

# Predict on validation set
y_pred = model.predict(X_val)

# Compute NWRMSLE
def nwrmsle(y_true, y_pred, weights):
    log_true = np.log1p(y_true)
    log_pred = np.log1p(y_pred)
    weighted_error = weights * (log_pred - log_true) ** 2
    return np.sqrt(weighted_error.sum() / weights.sum())

# Print NWRMSLE score
nwrmsle_score = nwrmsle(y_val, y_pred, w_val)
print(f"Validation NWRMSLE: {nwrmsle_score:.6f}")


In [ ]:
# Convert date columns into a single datetime column for test data
test_df["ds"] = pd.to_datetime(test_df[["year", "month", "day"]])

# Encode categorical features for test data
for col in ["store_nbr", "item_nbr", "city", "state", "family", "type", "cluster"]:
    test_df[col] = label_encoders[col].transform(test_df[col])  # Use stored encoders

# Merge perishable weights for test data
test_df = test_df.merge(items_df[["item_nbr", "weight"]], on="item_nbr", how="left")

# Define feature columns (same as used for training)
features = [
    "store_nbr", "item_nbr", "onpromotion", "is_holiday", 
    "year", "month", "weekday", "quarter", "perishable", "cluster"
]

# Extract features from test data
X_test = test_df[features]

# Predict on test data
y_test_pred = model.predict(X_test)

# Add predictions to test_df
test_df["unit_sales"] = y_test_pred

submission_df = test_df[["id", "unit_sales"]]

# Save the predictions to a CSV file
submission_df.to_csv("submission.csv", index=False)

# Print the first few rows of the submission (optional)
print(submission_df.head())
